In [1]:
import pandas as pd

In [2]:
# orders.csv
url = "https://drive.google.com/file/d/1DDuJ-kpWP2QWtKfz2i47FYmldVd1YvHI/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orders = pd.read_csv(path)
df_orders = orders.copy()

# orderlines.csv
url = "https://drive.google.com/file/d/1_2D1Ax74VtCdCoIrKc0IYf1fNovsGcHl/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orderlines = pd.read_csv(path)
df_orderlines = orderlines.copy()

# products.csv
url = "https://drive.google.com/file/d/1FwVKdNSDtCF6xf38GHbdpMSFR5ISnl-H/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products = pd.read_csv(path)
df_products = products.copy()

# brands.csv
url = "https://drive.google.com/file/d/17oeoOTq8I4XXmD8A7XdHIqoVy_AacSuZ/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
brands = pd.read_csv(path)
df_brands = brands.copy()

In [3]:
# Remove all the rows which have two decimal points.
two_dot_order_ids_list = df_orderlines.loc[df_orderlines.unit_price.str.contains("\d+\.\d+\.\d+"), "id_order"]
df_orderlines = df_orderlines.loc[~df_orderlines.id_order.isin(two_dot_order_ids_list)]


In [4]:
# Clean up the data
# orders
df_orders.drop_duplicates(inplace=True)
df_orders["created_date"] = pd.to_datetime(df_orders["created_date"])

# Focus on data which are completed
df_orders = df_orders.loc[df_orders.state == "Completed", :]

# orderlines
df_orderlines.drop_duplicates(inplace=True)
df_orderlines.rename(columns={"id_order": "order_id"}, inplace=True)
df_orderlines.drop(columns="product_id", axis=1, inplace=True)
df_orderlines["date"] = pd.to_datetime(df_orderlines["date"])

# products
df_products.drop_duplicates(inplace=True)
df_products.drop(columns="promo_price", axis=1, inplace=True)
df_products = df_products.dropna(subset=['price'])
df_products = df_products.loc[(~df_products.price.astype(str).str.contains("\d+\.\d+\.\d+"))&(~df_products.price.astype(str).str.contains("\d+\.\d{3,}")), :]
df_products["price"] = pd.to_numeric(df_products["price"])

# For the nan values in desc, the names are descriptive enough
df_products.loc[df_products['desc'].isna(), 'desc'] = df_products.loc[df_products['desc'].isna(), 'name']


In [5]:
# Extract ownerless orderlines
missing_order_ids = df_orderlines['order_id'][~df_orderlines['order_id'].isin(df_orders['order_id'])]
df_nirvana_orders = df_orderlines[df_orderlines['order_id'].isin(missing_order_ids)]
df_orderlines.drop(labels=df_nirvana_orders.index, inplace=True)


In [6]:
# Extract orderlines with unknown products
unknown_skus = df_orderlines["sku"][~df_orderlines["sku"].isin(df_products["sku"])]
df_nirvana_products = df_orderlines[df_orderlines["sku"].isin(unknown_skus)]
df_orderlines.drop(labels=df_nirvana_products.index, inplace=True)


In [9]:
# Saving data as .csv files to work with further
df_orders.to_csv("orders_cl.csv", index=False)
df_orderlines.to_csv("orderlines_cl.csv", index=False)
df_products.to_csv("products_cl.csv", index=False)
df_brands.to_csv("brands_cl.csv", index=False)

In [10]:
# Downloading
from google.colab import files

files.download("orders_cl.csv")
files.download("orderlines_cl.csv")
files.download("products_cl.csv")
files.download("brands_cl.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>